In [44]:
import os
import glob

import pandas as pd
import numpy as np
import rasterio as rio

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from scipy import stats
#import statannotations
from statannot import add_stat_annotation

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import imblearn as imb
from imblearn.under_sampling import RandomUnderSampler

import scipy

import rioxarray


In [45]:
# ps_20230616 = rioxarray.open_rasterio(
#     "/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2020/PScope/dis_band_2020/20200616_151605_21_2304_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif",
# )
# ps_20230616.name = 'ps_20230616'

In [46]:
# pscope_df = ps_20230616.to_dataframe()

In [47]:
# pscope_df['ps_20230616']

In [48]:
image = '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2020/PScope/dis_band_2020/20200616_151605_21_2304_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif'
with rio.open(image) as src:
        raster_arr = np.array(src.read())

In [60]:
raster_arr.shape

(11, 22, 41)

In [49]:
inc_class = raster_arr[10]*1000
sev_class = raster_arr[9]*1000
    

In [50]:
inc_class

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, 

In [59]:
inc_class = (inc_class > 25).astype(int)
inc_class.shape

(22, 41)

In [61]:
sev_class.shape

(22, 41)

In [65]:
# Try adding the severity and incidence classifications as bands to the dis_rasters

def add_classes(image, inc_band, sev_band) :
    '''Adds bands for severity and incidence classifications.
    "high" is coded as a 1
    "low" is coded as a 0
    
    image: (string) path to raster 
    inc_band: (int) band containing incidence values
    sev_band: (int) band containing severity values
    
    returns: array with original image bands + incidence and severity class bands
    '''
    
    with rio.open(image) as src:
        raster_arr = np.array(src.read())
        raster_arr = raster_arr
        
        inc_class = raster_arr[inc_band]
        sev_class = raster_arr[sev_band]
        
        # if incidence > 25%, enter "1", else "0"
        inc_class = (inc_class > 25).astype(int)
        
        # if severity > 10%, enter "1", else "0"
        sev_class = (sev_class > 10).astype(int)
        
        stacked_arr = np.vstack([raster_arr,
                                 inc_class.reshape(1,raster_arr.shape[1],raster_arr.shape[2]),
                                 sev_class.reshape(1,raster_arr.shape[1],raster_arr.shape[2])])
    
    # Save stacked array as raster
    with rio.open(image) as src:
        kwargs = src.meta
        band_ct = stacked_arr.shape[0]
        kwargs.update(dtype=rio.float32, count=band_ct)
        
        with rio.open(str(os.path.split(image)[0])+'/inc_sev_class_'+str(os.path.basename(image)), 'w', **kwargs) as dst:
            for b in range(stacked_arr.shape[0]):
                dst.write_band(b+1, stacked_arr[b].astype(rio.float32))

In [66]:
# Test function with 2020 PScope dis band images

ps_2020 = glob.glob('/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2020/PScope/dis_band_2020/20*.tif')
ps_2020

['/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2020/PScope/dis_band_2020/20200801_151354_03_2212_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif',
 '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2020/PScope/dis_band_2020/20200713_151457_44_2278_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif',
 '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2020/PScope/dis_band_2020/20200616_151605_21_2304_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif']

In [67]:
for img in ps_2020:
    add_classes(img, inc_band=10, sev_band=9)

In [68]:
ps_2021 = glob.glob('/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2021/PScope/dis_band_2021/20*.tif')
ps_2021

['/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2021/PScope/dis_band_2021/20210808_150651_57_2460_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif',
 '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2021/PScope/dis_band_2021/20210804_152058_95_222f_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif',
 '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2021/PScope/dis_band_2021/20210726_155527_27_227a_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif',
 '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2021/PScope/dis_band_2021/20210715_151038_58_2450_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif']

In [69]:
for img in ps_2021:
    add_classes(img, inc_band=10, sev_band=9)

In [70]:
ps_2022 = glob.glob('/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2022/PScope/dis_band_2022/20*.tif')
ps_2022

['/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2022/PScope/dis_band_2022/20220726_155410_80_2407_3B_AnalyticMS_SR_8b_clipped_clipped.tif_disease.tif',
 '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2022/PScope/dis_band_2022/20220801_150200_56_245c_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif',
 '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2022/PScope/dis_band_2022/20220828_151645_10_2262_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif_disease.tif',
 '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2022/PScope/dis_band_2022/20220630_150422_85_241d_3B_AnalyticMS_SR_8b_clipped_clipped.tif_disease.tif',
 '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2022/PScope/dis_band_2022/20220704_155057_15_240a_3B_AnalyticMS_SR_8b_clipped_clipped.tif_disease.tif',
 '/Users/kathleenkanaley/Desktop/grapes_from_space/data/images/2022/PScope/dis_band_2022/20220624_150204_01_241b_3B_AnalyticMS_SR_8b_clipped_clipped.

In [71]:
for img in ps_2022:
    add_classes(img, inc_band=10, sev_band=9)

In [2]:
# def img_to_df(img_path, cols):
#     with rio.open(img_path) as src:
#         as_array = src.read()
#         band_arrs = []
        
#         for i in np.arange(src.count):
#             band_arr = as_array[i].flatten()
#             band_arr = band_arr/10000
            
#             band_arrs.append(band_arr)
        
#         df = pd.DataFrame(band_arrs,cols).T
#         df['Date'] = img_path.split('/')[-1].split('_')[0]
        
#         # Optional: drops entries with no severity rating. Comment out if unnecessary.
#         no_nans = df.dropna(subset=['total_dis_sev']) 
        
#         return no_nans

In [3]:
# def add_vis(df):
#     df['pri']= (df['green']- df['greenI'])/(df['green']+df['greenI'])
#     df['ndre'] = (df['nir'] - df['rede']) / (df['nir'] + df['rede'])
#     df['tcari'] = 3 * (( (df['rede'] - df['red']) - 0.2 * (df['rede'] - df['green']) ) * (df['rede'] / df['red']))
#     df['ndvi']= (df['nir']-df['red'])/(df['nir']+df['red'])
#     df['evi']= 2.5*(df['nir']-df['red'])/(df['nir']+6*df['red']-7.5*df['blue']+1)
#     df['savi']= 1.5*(df['nir']-df['red'])/(df['nir']+df['red']+0.5)
#     df['arvi']=(df['nir']-(2*df['red']-df['blue']))/(df['nir']+(2*df['red']-df['blue']))
#     df['green_red'] = (df['green']-df['red'])/(df['green']+df['red'])

#     return df

In [4]:
# data_dir = '/Users/kathleenkanaley/Desktop/grapes_from_space/data/'
# dis_ras_20 = glob.glob(data_dir +'images/2020/PScope/dis_band_*/*.tif')
# dis_ras_21 = glob.glob(data_dir +'images/2021/PScope/dis_band_*/*.tif')
# dis_ras_22 = glob.glob(data_dir +'images/2022/PScope/dis_band_*/*.tif')

In [5]:
# cols = ['c_blue','blue','greenI', 'green', 'yellow', 'red', 'rede', 'nir','total_dis_sev','dm_sev', 'dm_inc']

# # 2020

# dfs_2020 = [img_to_df(dis_ras_20[i], cols) for i in np.arange(len(dis_ras_20))]
# df_2020 = pd.concat(dfs_2020)

# df_2020['total_dis_sev'] = df_2020['total_dis_sev']*10000
# df_2020['dm_sev'] = df_2020['dm_sev']*10000
# df_2020['dm_inc'] = df_2020['dm_inc']*10000

# df_2020['inc_class'] = 'low'
# df_2020.loc[df_2020['dm_inc']>25,'inc_class']= 'high'
    
# df_2020['severity_class'] = 'low'
# df_2020.loc[df_2020['dm_sev']>10,'severity_class']= 'high'

# vis_2020 = add_vis(df_2020)


# #2021
# dfs_2021 = [img_to_df(dis_ras_21[i], cols) for i in np.arange(len(dis_ras_21))]
# df_2021 = pd.concat(dfs_2021)

# df_2021['total_dis_sev'] = df_2021['total_dis_sev']*10000
# df_2021['dm_sev'] = df_2021['dm_sev']*10000
# df_2021['dm_inc'] = df_2021['dm_inc']*10000

# df_2021['inc_class'] = 'low'
# df_2021.loc[df_2021['dm_inc']>25,'inc_class']= 'high'
    
# df_2021['severity_class'] = 'low'
# df_2021.loc[df_2021['dm_sev']>10,'severity_class']= 'high'

# vis_2021 = add_vis(df_2021)
# vis_2021 = vis_2021[vis_2021['Date']!='20210804']

# #2022
# dfs_2022 = [img_to_df(dis_ras_22[i], cols) for i in np.arange(len(dis_ras_22))]
# df_2022 = pd.concat(dfs_2022)

# df_2022['total_dis_sev'] = df_2022['total_dis_sev']*10000
# df_2022['dm_sev'] = df_2022['dm_sev']*10000
# df_2022['dm_inc'] = df_2022['dm_inc']*10000

# df_2022['inc_class'] = 'low'
# df_2022.loc[df_2022['dm_inc']>25,'inc_class']= 'high'
    
# df_2022['severity_class'] = 'low'
# df_2022.loc[df_2022['dm_sev']>10,'severity_class']= 'high'

# vis_2022 = add_vis(df_2022)
# vis_2022.columns

Index(['c_blue', 'blue', 'greenI', 'green', 'yellow', 'red', 'rede', 'nir',
       'total_dis_sev', 'dm_sev', 'dm_inc', 'Date', 'inc_class',
       'severity_class', 'pri', 'ndre', 'tcari', 'ndvi', 'evi', 'savi', 'arvi',
       'green_red'],
      dtype='object')

In [ ]:
# def run_rf(df, expl_vars, resp_var, SPLIT_RND_SEED):
#     X = df[expl_vars]
#     y = df[resp_var]
    
#     rus = RandomUnderSampler(random_state=2020)
#     X_rus, y_rus = rus.fit_resample(X, y)

# # Split dataset into training set and test set

#     X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus,
#                                                     test_size=0.3, 
#                                                     random_state=SPLIT_RND_SEED,
#                                                     stratify = y_rus)  # 70% training and 30% test
#     mm = MinMaxScaler()
#     X_train_scaled = mm.fit_transform(X_train)
#     X_test_scaled = mm.transform(X_test)
    
    
#     from sklearn.ensemble import RandomForestClassifier
    
#     # Modify with best parameters  
#     rf = RandomForestClassifier()
        
        
# #                                 n_estimators = 500, 
# #                                 oob_score=True, 
# #                                 #random_state=rand_state, # random number to be used, needed to reproduce the same result
# #                                 verbose=False)
    
#     rf_model = rf.fit(X_train_scaled, y_train)
    
#     y_pred = rf_model.predict(X_test_scaled)
    
#     from sklearn.metrics import accuracy_score
#     accuracy = accuracy_score(y_test, y_pred)
    
#     from sklearn.metrics import precision_recall_fscore_support
    
#     prec, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred)
  
    
#     from sklearn.metrics import cohen_kappa_score
        
#     cohen_score = cohen_kappa_score(y_test, y_pred)    

#     label_0 = np.unique(y_pred)[0]
#     label_1 = np.unique(y_pred)[1]
    
#     metrics = {
#         'cohen_kappa': [cohen_score], 
#         'accuracy_score': [accuracy],
#         'precision {}'.format(label_0): [prec[0]],
#         'precision {}'.format(label_1): [prec[1]],
#         'recall {}'.format(label_0): [recall[0]],
#         'recall {}'.format(label_1): [recall[1]],
#         'fscore {}'.format(label_0): [fscore[0]],
#         'fscore {}'.format(label_1): [fscore[1]],

#     }
#     metrics_df = pd.DataFrame(data=metrics)
    
#     return metrics_df